<a href="https://colab.research.google.com/github/AlexandreMinato/Investimentos_FII/blob/main/Analises_FII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import logging
import requests
import numpy as np
import pandas as pd

# Capturando os dados

In [ ]:
url = 'https://www.fundsexplorer.com.br/ranking'
dados = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
list_obj = pd.read_html(dados.text, attrs={'id': 'table-ranking'})
df = list_obj[0]

In [ ]:
# redefinindo nome das colunas
df.columns = ['ativo', 'setor', 'preco', 'liquidez_dia', 'dividendo', 'yeld', 'yeld_3m_acululado', 'yeld_6m_acululado', 'yeld_12m_acululado', 'yeld_3m_media', 'yeld_6m_media', 'yeld_12m_media', 'yeld_ano', 'variacao_preco', 'rent_periodo', 'rent_acumulada', 'patrimonio_liq', 'vpa', 'p_vpa', 'dy_patrimonial', 'variacao_patrimonial', 'rent_patr_periodo', 'rent_acumulada_periodo', 'vacancia_fisica', 'vacancia_financeira', 'quantidade_ativos']

In [ ]:
df.head()

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
0,FIVN11,Shoppings,"R$ 3,09",31730.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%",NaN,"0,00%","0,00%","0,00%","R$ 64.832.865,32","R$ 6,89",45.0,NaN,NaN,NaN,NaN,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,"R$ 14,70",44041.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,68%","0,68%","1,14%","R$ 439.665.897,40","R$ 11,45",128.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 6,88",993.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","1,10%","1,10%","-37,83%","R$ 25.472.636,57","R$ 14,21",48.0,NaN,NaN,NaN,NaN,"0,00%","100,00%",1
3,ALMI11,Lajes Corporativas,"R$ 1.180,00",31.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","2,27%","2,27%","-28,29%","R$ 229.423.903,28","R$ 2.063,59",57.0,NaN,NaN,NaN,NaN,"68,15%",NaN,1
4,DOMC11,Lajes Corporativas,"R$ 439,00",42.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","4,35%","4,35%","-10,00%","R$ 264.351.778,61","R$ 108,17",406.0,NaN,NaN,NaN,NaN,"23,44%",NaN,1


#Normalizando as colunas



In [ ]:
def identificar_conteudo_colunas(df, item):
    
    return [df[column].name for column in df if df[column].astype(str).str.contains(item).any()]

In [ ]:
#Converter financeiro
coluna_monetario = identificar_conteudo_colunas(df, '\$')

for column_name in coluna_monetario:
    df[column_name] = df[column_name].str.replace('.', '')
    df[column_name] = df[column_name].str.replace(',', '.')
    df[column_name] = df[column_name].str.replace('[R\$ ,]', '', regex=True).astype(float)

In [ ]:
# Converter as colunas de percentual em numericas
coluna_percentual = identificar_conteudo_colunas(df, '\%')
for column_name in coluna_percentual:
    df[column_name] = df[column_name].str.replace(',', '.')
    df[column_name] = df[column_name].str.replace('[%,]', '', regex=True).astype(float)
    df[column_name] = df[column_name]/100

In [ ]:
# Normaliza o setor
df['setor'] = df['setor'].str.replace('\s+', '_')
df['setor'] = df['setor'].str.replace(r'[^\w\s]+', '_')
df['setor'] = df['setor'].str.lower()
df['setor'] = df['setor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Iniciando as analises


In [ ]:
df.setor.unique()

array(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas',
       'hibrido', 'outros', 'hotel', 'residencial', 'logistica',
       'hospital'], dtype=object)

In [ ]:
df.ativo.nunique()

238

In [ ]:
# Convert p/vpa to the correct base
df['p_vpa'] = df['p_vpa']/100
# Checking if is it correct
df[df['ativo'] == 'VRTA11']

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
106,VRTA11,titulos_e_val__mob_,112.38,17785.0,0.9,0.0078,0.0235,0.0461,0.0839,0.0078,0.0077,0.007,0.0313,0.006,0.0138,0.0304,1.010653e+09,101.07,1.11,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
df.describe()

,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
count,228.000000,2.280000e+02,238.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,212.000000,222.000000,222.000000,222.000000,2.380000e+02,238.000000,228.00,4.000000,4.000000,4.000000,4.000000,116.000000,25.000000,238.000000
mean,519.575088,2.267750e+04,1.316261,0.007403,0.018024,0.032010,0.051234,0.006009,0.005334,0.004271,0.025496,-0.014414,-0.007442,-0.031715,5.149004e+08,662.396429,inf,0.006950,0.002075,0.009000,0.022475,0.155532,0.126372,3.331933
std,5642.306143,1.006695e+05,4.445995,0.024198,0.026794,0.033587,0.051449,0.008929,0.005599,0.004289,0.028759,0.103475,0.103215,0.123874,7.104465e+08,6211.667336,NaN,0.001109,0.003292,0.003562,0.004882,0.228208,0.232114,7.999150
min,0.780000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.946500,-0.946200,-0.945900,-5.574880e+06,-7.350000,-1.72,0.005900,0.000100,0.006500,0.016200,0.000000,0.000000,0.000000
25%,82.282500,2.182500e+02,0.400000,0.004100,0.009200,0.015675,0.000000,0.003050,0.002625,0.000000,0.016075,-0.022675,-0.016175,-0.050900,1.030673e+08,93.750000,0.84,0.006200,0.000475,0.006800,0.019875,0.000000,0.000000,0.000000
50%,99.120000,2.483000e+03,0.630000,0.006100,0.016700,0.032400,0.055600,0.005550,0.005400,0.004600,0.022450,-0.000500,0.000400,-0.009900,2.621177e+08,99.915000,0.97,0.006750,0.000600,0.007650,0.023350,0.057350,0.036000,1.000000
75%,111.545000,1.481425e+04,0.900000,0.007675,0.022200,0.043675,0.077175,0.007400,0.007275,0.006400,0.030100,0.013275,0.018775,0.023900,5.922617e+08,109.795000,1.04,0.007500,0.002200,0.009850,0.025950,0.211000,0.123000,3.000000
max,85227.760000,1.336487e+06,52.740000,0.363200,0.379900,0.409600,0.452200,0.126600,0.068300,0.037700,0.392400,0.327300,0.330300,0.410700,4.852707e+09,93717.970000,inf,0.008400,0.007000,0.014200,0.027000,1.000000,1.000000,72.000000


In [ ]:
df.setor.unique()

array(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas',
       'hibrido', 'outros', 'hotel', 'residencial', 'logistica',
       'hospital'], dtype=object)

In [ ]:
# filtrando setores
df = df[df['setor'].isin(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas', 'hibrido', 'logistica'])].copy()

In [ ]:
df.setor.unique()

array(['shoppings', 'titulos_e_val__mob_', 'lajes_corporativas',
       'hibrido', 'logistica'], dtype=object)

In [ ]:
df.head()

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos
0,FIVN11,shoppings,3.09,31730.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0000,0.0000,0.0000,6.483287e+07,6.89,0.45,NaN,NaN,NaN,NaN,0.5600,NaN,1
1,BZLI11,titulos_e_val__mob_,14.70,44041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0068,0.0068,0.0114,4.396659e+08,11.45,1.28,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,lajes_corporativas,6.88,993.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0110,0.0110,-0.3783,2.547264e+07,14.21,0.48,NaN,NaN,NaN,NaN,0.0000,1.0,1
3,ALMI11,lajes_corporativas,1180.00,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0227,0.0227,-0.2829,2.294239e+08,2063.59,0.57,NaN,NaN,NaN,NaN,0.6815,NaN,1
4,DOMC11,lajes_corporativas,439.00,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0435,0.0435,-0.1000,2.643518e+08,108.17,4.06,NaN,NaN,NaN,NaN,0.2344,NaN,1


#Condicoes para filtrar os FIIs

In [ ]:
#Preco aceitavel para pagamento, considerando cota e valor patrimonial
df['preco_justo'] = np.where((df['p_vpa'] > 0.9) & (df['p_vpa'] < 1.1) , True, False)

setores_considerar = ['shoppings', 'lajes_corporativas', 'logistica']

# Diversificacao do papel
diversificacoes_ativos = [
    # Condicoes do setor com mais de 5 ativos
    (df['setor'].isin(setores_considerar)) & 
    (df['quantidade_ativos'] > 5),
    # Condicoes do setor com menos de 5 ativos
    (df['setor'].isin(setores_considerar)) &
    (df['quantidade_ativos'] < 5),
    # setores
    (~df['setor'].isin(setores_considerar))
]
diversificacoes_aceitaveis = [True, False, True]
df['ativos_diversificados'] = np.select(diversificacoes_ativos, diversificacoes_aceitaveis, default=None)

# imovel com vacancia aceitavel
df['vacancia_aceitavel'] = np.where((df['vacancia_fisica'] < 0.1) | (df['vacancia_fisica'].isnull()) | (df['vacancia_fisica'].isnull()) , True, False)

# liquidez
df['liquidez'] = np.where(df['liquidez_dia'] > 20000, True, False)

# que paga bons dividendos
df['dividendos_minimos'] = np.where(df['yeld_12m_acululado'] > 0.05, True, False)

In [ ]:
condicoes_ativos = (
    df.preco_justo &
    df.ativos_diversificados &
    df.vacancia_aceitavel &
    df.liquidez &
    df.dividendos_minimos
)
df_results = df.loc[condicoes_ativos].sort_values(by=['yeld_12m_acululado', 'liquidez_dia'], ascending=False)

### Resultado
*Os papéis abaixo atendem aos filtros de diversificação e preço justo.

In [ ]:
df_results

,ativo,setor,preco,liquidez_dia,dividendo,yeld,yeld_3m_acululado,yeld_6m_acululado,yeld_12m_acululado,yeld_3m_media,yeld_6m_media,yeld_12m_media,yeld_ano,variacao_preco,rent_periodo,rent_acumulada,patrimonio_liq,vpa,p_vpa,dy_patrimonial,variacao_patrimonial,rent_patr_periodo,rent_acumulada_periodo,vacancia_fisica,vacancia_financeira,quantidade_ativos,preco_justo,ativos_diversificados,vacancia_aceitavel,liquidez,dividendos_minimos
189,RECR11,titulos_e_val__mob_,101.40,52859.0,1.10,0.0100,0.0280,0.0673,0.1232,0.0093,0.0112,0.0103,0.0407,0.0084,0.0185,0.0227,1.355698e+09,95.23,1.06,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
196,CVBI11,titulos_e_val__mob_,104.60,32898.0,1.00,0.0094,0.0287,0.0732,0.1171,0.0096,0.0122,0.0098,0.0402,0.0009,0.0104,0.0211,5.961533e+08,100.08,1.05,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
192,CPTS11,titulos_e_val__mob_,99.80,103882.0,1.05,0.0104,0.0307,0.0615,0.1064,0.0102,0.0102,0.0089,0.0408,-0.0231,-0.0130,0.0204,1.749247e+09,95.18,1.05,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
75,BCRI11,titulos_e_val__mob_,115.01,35865.0,1.51,0.0127,0.0316,0.0600,0.1040,0.0105,0.0100,0.0087,0.0392,0.0442,0.0574,0.0473,4.698016e+08,106.77,1.08,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
114,KNIP11,titulos_e_val__mob_,109.50,49924.0,1.00,0.0088,0.0280,0.0598,0.0945,0.0093,0.0100,0.0079,0.0392,-0.0254,-0.0168,0.0328,4.852707e+09,102.95,1.06,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
63,KNHY11,titulos_e_val__mob_,111.79,35745.0,1.20,0.0105,0.0313,0.0638,0.0930,0.0104,0.0106,0.0078,0.0439,-0.0020,0.0085,0.0611,1.095864e+09,102.50,1.09,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
122,MXRF11,hibrido,10.58,1336487.0,0.08,0.0075,0.0226,0.0425,0.0804,0.0075,0.0071,0.0067,0.0301,0.0250,0.0327,0.0344,2.272793e+09,10.02,1.06,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
199,RBRF11,titulos_e_val__mob_,89.13,24166.0,0.60,0.0065,0.0194,0.0423,0.0798,0.0065,0.0070,0.0066,0.0255,-0.0169,-0.0105,-0.0506,1.257992e+09,91.95,0.97,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
140,VINO11,lajes_corporativas,60.66,42512.0,0.45,0.0074,0.0219,0.0414,0.0792,0.0073,0.0069,0.0066,0.0290,-0.0142,-0.0070,-0.0172,7.607044e+08,60.02,1.01,NaN,NaN,NaN,NaN,0.037,NaN,9,True,True,True,True,True
202,MCCI11,titulos_e_val__mob_,103.02,46375.0,0.70,0.0069,0.0204,0.0389,0.0740,0.0068,0.0065,0.0062,0.0272,-0.0039,0.0029,0.0170,8.859323e+08,99.24,1.04,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True,True,True
